## Описание данных 

`date` – дата совершения события

`event` - событие

* `app_install` – установка приложения
* `app_start` – открыть приложения
* `registration` – зарегистрироваться 
* `search` – перейти на страницу поиска товаров (каталог)
* `open_item` – открыть товар
* `choose_item` – отправить товар в корзину
* `tap_basket` – перейти в корзину
* `purchase` – подтверждение покупки

`gender` – пол пользователя

`os_name` – платформа пользователя

`city` – город пользователя

`device_id` – идентификатор устройства пользователя

`urm_source` – канал, с которого пришел пользователь

* `yandex-direct` – Яндекс директ
* `google_ads` – реклама в Google
* `vk_ads` – реклама в ВК
* `instagram_ads` – реклама в instagram
* `facebook_ads` – реклама в facebook
* `referral` – акция «приведи друга»

Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
`purchase_sum` – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [143]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [181]:
data = pd.read_csv('/Users/pavel/Desktop/karpov/10 МОДУЛЬ/3 KC_case_data .csv')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [182]:
data.date = pd.to_datetime(data['date'])
data.query("date.dt.month == 2").device_id.nunique()

75032

In [183]:
data.query("date.dt.month == 1 and event == 'app_install'").event.count()

80297

In [184]:
installs = data[data['event'] == 'app_install'][['date', 'device_id']]
installs = installs.drop_duplicates('device_id')
installs.rename(columns = {'date' : 'install_date'}, inplace = True)
data = data.merge(installs, how = 'left', on = 'device_id')
purchase = data.query("event == 'purchase'")[['date', 'device_id']]
purchase.rename(columns = {'date' : 'first_purchase'}, inplace = True)
data = data.merge(purchase, how = 'left', on = 'device_id')
data.to_csv("KC_resuls.csv", index = False)
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,first_purchase
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-09
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-11
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-16
...,...,...,...,...,...,...,...,...,...,...
10652408,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31
10652409,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31
10652410,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31
10652411,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31


In [185]:
data['date_diff'] = data.first_purchase - data.install_date
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,first_purchase,date_diff
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6 days
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-09,8 days
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0 days
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-11,10 days
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-16,15 days


In [186]:
data.query("date_diff.dt.days <= 7").device_id.nunique()

46674

In [187]:
data.device_id.nunique()

190884

In [188]:
data.query("date_diff.dt.days <= 7").device_id.nunique() / data.device_id.nunique()

0.24451499339913246

In [189]:
all_device = data.groupby("install_date", as_index = False).agg({'device_id': 'nunique'})
all_device

,install_date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [190]:
device_7 = data.query("date_diff.dt.days <= 7").groupby("install_date", as_index = False).agg({'device_id': 'nunique'})
device_7

,install_date,device_id
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587
...,...,...
86,2020-03-27,277
87,2020-03-28,209
88,2020-03-29,223
89,2020-03-30,204


In [191]:
(device_7.device_id / all_device.device_id * 100).sort_values(ascending = False)

0     39.340598
8     39.185393
14    38.283063
13    38.140344
1     37.722646
        ...    
77    20.068318
88    19.964190
81    19.904837
87    19.156737
65    12.692671
Name: device_id, Length: 91, dtype: float64

In [192]:
data = pd.read_csv('/Users/pavel/Desktop/karpov/10 МОДУЛЬ/3 KC_case_data .csv')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [193]:
data.query("event == 'app_install'").groupby('utm_source', as_index = False).agg({'device_id': 'nunique'})

,utm_source,device_id
0,-,32460
1,facebook_ads,13916
2,google_ads,26286
3,instagram_ads,20096
4,referal,9282
5,vk_ads,23189
6,yandex-direct,29368


In [194]:
data.groupby('event').device_id.nunique().sort_values()

event
purchase        70875
register        78310
tap_basket     125414
app_install    154597
choose_item    155691
search         184488
app_start      190884
Name: device_id, dtype: int64

In [195]:
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [202]:
register = data.query("event == 'register'")[['device_id', 'date']].sort_values('date').drop_duplicates('device_id')
register

,device_id,date
2669658,294193,2020-01-01
2670356,21780,2020-01-01
2670357,2470180,2020-01-01
2670358,3735668,2020-01-01
2670359,11372938,2020-01-01
...,...,...
2747651,32856164,2020-03-31
2747650,228794,2020-03-31
2747649,20089663,2020-03-31
2747661,5174438,2020-03-31


In [204]:
first_registration = data[data['event'] == 'register'].sort_values('date').drop_duplicates('device_id')
data['first_registration'] = data['device_id'].map(first_registration.set_index('device_id')['date'])

In [210]:
already_reg = data[data['first_registration'] < data['date']]
already_reg.groupby('event').device_id.nunique()

event
app_start      40991
choose_item    37926
purchase       24880
search         40482
tap_basket     34517
Name: device_id, dtype: int64

In [236]:
print('Конверсия в поиск из открытия', 
np.round(already_reg.query("event == 'search'").device_id.drop_duplicates().count() / \
already_reg.query("event == 'app_start'").device_id.drop_duplicates().count() * 100,2))

Конверсия в поиск из открытия 98.76


In [240]:
print('Конверсия в добавление товара из поиск', 
np.round(already_reg.query("event == 'choose_item'").device_id.drop_duplicates().count() / \
already_reg.query("event == 'search'").device_id.drop_duplicates().count() * 100,2))

Конверсия в добавление товара из поиск 93.69


In [242]:
print('Конверсия в переход в корзину из добавление товара', 
np.round(already_reg.query("event == 'tap_basket'").device_id.drop_duplicates().count() / \
already_reg.query("event == 'choose_item'").device_id.drop_duplicates().count() * 100,2))

Конверсия в переход в корзину из добавление товара 91.01


In [244]:
print('Конверсия в покупку из перехода в корзину', 
np.round(already_reg.query("event == 'purchase'").device_id.drop_duplicates().count() / \
already_reg.query("event == 'tap_basket'").device_id.drop_duplicates().count() * 100,2))

Конверсия в покупку из перехода в корзину 72.08


In [251]:
purchase = data.query("event == 'purchase'")[['date', 'purchase_sum', 'device_id', 'utm_source']]
purchase = purchase.sort_values('date').drop_duplicates('device_id')
purchase

,date,purchase_sum,device_id,utm_source
7691,2020-01-01,265.5,6081155,-
8289,2020-01-01,292.0,1955306,yandex-direct
8290,2020-01-01,664.0,20176327,facebook_ads
8291,2020-01-01,337.0,28603960,-
8292,2020-01-01,502.0,14028316,referal
...,...,...,...,...
2506842,2020-03-31,670.0,32122725,-
2506845,2020-03-31,407.0,23188703,instagram_ads
2506848,2020-03-31,149.5,25732558,referal
2506852,2020-03-31,442.5,17778508,referal


In [261]:
purchase.query("utm_source != '-'").groupby('utm_source', as_index = False).agg({'purchase_sum': 'median'}).\
    sort_values('purchase_sum', ascending = False)

,utm_source,purchase_sum
3,referal,395.5
2,instagram_ads,393.5
4,vk_ads,393.0
5,yandex-direct,392.5
1,google_ads,390.5
0,facebook_ads,389.0


In [276]:
romi_table = data.query("utm_source != '-'").groupby('utm_source', as_index = False).purchase_sum.sum()
romi_table

,utm_source,purchase_sum
0,facebook_ads,12249901.0
1,google_ads,12868276.0
2,instagram_ads,14546969.0
3,referal,8837044.5
4,vk_ads,16389652.5
5,yandex-direct,13915368.0


In [309]:
print('ROMI Facebook',(romi_table.query("utm_source == 'facebook_ads'").purchase_sum / 8590498 - 1)\
      .mul(100).round(2).values[0], '$')
print('ROMI Google',(romi_table.query("utm_source == 'google_ads'").purchase_sum / 10534878 - 1)\
      .mul(100).round(2).values[0], '$')
print('ROMI Instagram',(romi_table.query("utm_source == 'instagram_ads'").purchase_sum / 8561626 - 1)\
      .mul(100).round(2).values[0], '$')
print('ROMI Vk',(romi_table.query("utm_source == 'vk_ads'").purchase_sum / 9553531 - 1)\
      .mul(100).round(2).values[0], '$')
print('ROMI Yandex-direct',(romi_table.query("utm_source == 'yandex-direct'").purchase_sum / 10491707 - 1)\
      .mul(100).round(2).values[0], '$')

ROMI Facebook 42.6 $
ROMI Google 22.15 $
ROMI Instagram 69.91 $
ROMI Vk 71.56 $
ROMI Yandex-direct 32.63 $
